# Running the ZPVC code for an NMR isotropic value from a Gaussian calculation

Requirements:

 - Need the gradient and property (NMR) calculations of all the diaplaced structures.
 
The output files must all be in some readable directory so we can use the `vibrav.util.open_files.get_data()` function. In this example the gradients and properties are calculated separately. It may be the case that for the user the gradients and properties can be calculated in the same calculation simplifying things slightly. One would still need the gradient and property values.

In [1]:
from vibrav.base import resource
import tarfile
import os
import shutil
import numpy as np

Extract the data files from the resource data files.

In [2]:
with tarfile.open(resource('g16-nitromalonamide-zpvc-data.tar.xz'), 'r:xz') as tar:
    tar.extractall()
parent = os.getcwd()
os.chdir('g16-nitromalonamide-zpvc-data')

## Get the gradient and property data frames.

We get only the isotropic values of the NMR shielding tensor since we are interested in the observable quantity. In addition, we must specify which atom the property is to be calculated for.

In [3]:
from vibrav.util.open_files import get_all_data
from exatomic import gaussian

In [4]:
grad = get_all_data(cls=gaussian.Output, path='output', property='gradient',
                    f_start='nitromal-grad', f_end='.out')
prop = get_all_data(cls=gaussian.Output, path='output', property='nmr_shielding',
                    f_start='nitromal-prop', f_end='.out')
prop = prop.groupby('atom').get_group(0)[['isotropic', 'file']]

Check if any of the gradient or property calculations did not finish correctly. This is also done in the ZPVC code, however, this can give a better idea of which are actually missing.

In [5]:
print("Missing property calculations:")
print(np.setdiff1d(range(79), prop.file.values))
print("Missing gradient calculations:")
print(np.setdiff1d(range(79), grad.file.values))

Missing property calculations:
[]
Missing gradient calculations:
[]


## Run the ZPVC code for temperatures `[0, 100, 200, 300, 400, 600]`.

In [6]:
from vibrav.zpvc import ZPVC

In [7]:
zpvc = ZPVC(config_file='va.conf')

In [8]:
zpvc.zpvc(gradient=grad, property=prop, temperature=[0, 100, 200, 300, 400, 600])

/home/herbertl/github/vibrav/vibrav/zpvc/zpvc.py:175: Warning: Negative force constants have been calculated for frequencies 5, 11, 14, 22, 30, 31, 33 be wary of results
  Warning)
/home/herbertl/github/vibrav/vibrav/zpvc/zpvc.py:177: RuntimeWarning: invalid value encountered in sqrt
  frequencies = np.sqrt(vqi).reshape(snmodes,)*Energy['Ha', 'cm^-1']
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Print out the main results

In [9]:
zpvc.zpvc_results

,property,zpvc,zpva,tot_anharm,tot_curva,temp
0,13.9329,-2.412468,11.520432,-0.363847,-2.048620,0
1,13.9329,-2.743256,11.189644,-0.310899,-2.432358,100
2,13.9329,-3.170739,10.762161,-0.215905,-2.954834,200
3,13.9329,-3.701864,10.231036,-0.127152,-3.574713,300
4,13.9329,-4.331624,9.601276,-0.045316,-4.286308,400
5,13.9329,-5.782577,8.150323,0.102157,-5.884735,600


Clean up the resuorce data files.

In [10]:
os.chdir(parent)
shutil.rmtree('g16-nitromalonamide-zpvc-data')